In [2]:
import tensorflow as tf
# 导入数据
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# 导入 MNIST 数据集，labels 使用 one-hot 编码
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# 设置每个批次大小
batch_size = 100
# 计算样本一共有都少批次
n_batch = mnist.train.num_examples // batch_size

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
# 定义函数
# 
# 初始化权重
def weigth_variable(shape):
    initial = tf.truncated_normal(shape=shape, mean=0, stddev=1.0) # 生成截断正态分布
    return tf.Variable(initial_value=initial)

# 初始化偏置
def biase_variable(shape):
    initial = tf.constant(value=0.1, shape=shape)
    return tf.Variable(initial)

In [5]:
# 卷积层
def conv2d(x, W):
#     input 为输入数据，tensor of shape [batch, in_height, in_width, in_channels]
#                                          [批次大小，长， 宽， 通道数]
#     filter 为卷积核，tensor of shape [filter_height, filter_width, inchannels, out_channels]
#                                 [卷积核高， 卷积核宽， 通道数，卷积核个数（本层）]
#     strides 为步长，must strides[0]=stride[3]=1,strides[1] 表示 x 方向步长，strides[3] 表示 y 方向步长
#     padding 取值 'VALID' \ 'SAME'
    return  tf.nn.conv2d(input=x, filter=W, strides=[1, 1, 1, 1], padding='SAME')

# 池化层
def max_pool_2x2(x):
#     value 输入数据
#     ksize 池化核大小 ksize[0]=ksize[3] 必须为 0，剩下为 x 方向，y 方向
#     strides 步长，同上
    return tf.nn.max_pool(value=x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [6]:
# 定义两个 placeholder
x = tf.placeholder(dtype=tf.float32, shape=(None, 784)) # 图像大小 （28*28）
y = tf.placeholder(dtype=tf.float32,shape=(None, 10))

# 改变 x 的格式，将 x 转换为 4D 的向量 [batch, in_heigth, in_width, in_channels]
x_image = tf.reshape(tensor=x, shape=(-1, 28, 28, 1))

# 第一层
# 
# 初始化卷积核的权值核偏置
# 
# shape 为输入 tensor 的 shape [height, width, channel, filter_num]
W_conv1 = weigth_variable(shape=(3, 3, 1, 32))

# 每一个卷积核设置一个偏置
b_conv1 = biase_variable(shape=(1, 32))

# 激活函数
h_conv1 = tf.nn.relu(features=(conv2d(x=x_image, W=W_conv1) + b_conv1))

# 池化
h_pool = max_pool_2x2(h_conv1) # 进行 max_pool 池化


# 第二层
# 
# 初始化卷积核的权值核偏置
# 
# 上一层的卷积核为 32，固这层的输入通道数为 32
W_conv2 = weigth_variable(shape=(3, 3, 32, 64))

# 每一个卷积核设置一个偏置
b_conv2 = biase_variable(shape=(1, 64))

# 激活函数
h_conv2 = tf.nn.relu(features=(conv2d(x=h_pool, W=W_conv2) + b_conv2))

# 池化
h_poo2 = max_pool_2x2(h_conv2) # 进行 max_pool 池化

In [8]:
# (28 * 28) 的图像第一次卷积为后还是 (28 * 28)，池化使其缩小到 (14 * 14)
#                                            原因为，padding='SAME',进行填充补 0，所以运用公式有向上取整的要求
# (14 * 14) 的特征图（feature）,卷积操作不改变大小，池化变为 (7 * 7)
# 第二曾输出的 tensor 的 shape 为 (7, 7, 64)

# 扁平化处理（以下要连接全连接层）
h_poo2_flat = tf.reshape(tensor=h_poo2, shape=(-1, 7 * 7 * 64))

# 全连接层
# 
# 第一全连接层
# 
# 初始化全连接的权值和偏置
W_fc1 = weigth_variable(shape=(7 * 7 * 64, 1024)) # 上一层的输出为 (7 * 7 * 64)，这一层全连接层设置 1024 个神经元
b_fc1 = biase_variable(shape=(1, 1024))

h_fc1 = tf.nn.relu(features=tf.matmul(a=h_poo2_flat, b=W_fc1) + b_fc1)

# 使用 Dropout 降低过拟合风险
keep_prob = tf.placeholder(dtype=tf.float32)
h_fc1_dropout = tf.nn.dropout(x=h_fc1, keep_prob=keep_prob)

# 第二个全连接层
# 
# 初始化全连接的权值和偏置
W_fc2 = weigth_variable(shape=(7 * 7 * 64, 10)) # 上一层的输出为 (None, 1024)，这一层全连接层设置 10 个神经元,用于分类
b_fc2 = biase_variable(shape=(1, 10))

# 计算输出，使用 softmat 激活函数
prediction = tf.nn.softmax(logits=tf.matmul(a=h_poo2_flat, b=W_fc2) + b_fc2)

print(y.shape)
print(prediction.shape)

(?, 10)
(?, 10)


In [9]:
# 定义损失函数，使用交叉熵
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))

# 定义优化器，使用 Adam
train = tf.train.AdamOptimizer().minimize(loss=cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [10]:
# 将结果存放在 bool 列表中
# 
correct_prediction = tf.equal(x=tf.argmax(input=y, axis=1), y=tf.argmax(input=prediction, axis=1))

# 计算准确率
accuracy = tf.reduce_mean(tf.cast(x=correct_prediction, dtype=tf.float32))

In [11]:
# 初始化变量
init_op = tf.global_variables_initializer()
epochs = 20

In [12]:
# 计算图
# 
with tf.Session() as sess:
    sess.run(init_op)
    for epoch in range(epochs):
        for _ in range(n_batch):
            x_data, y_data = mnist.train.next_batch(batch_size=batch_size)
            loss, _ = sess.run(fetches=[cross_entropy, train], feed_dict={x: x_data, y: y_data, keep_prob: 0.7})
            
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1})
        
#         打印
        print('Epoch=' + str(epoch), '  loss=' + str(loss), ' accuracy=' + str(acc))

Epoch=0   loss=2.231147  accuracy=0.295
Epoch=1   loss=2.0511503  accuracy=0.3682
Epoch=2   loss=2.041084  accuracy=0.3918
Epoch=3   loss=2.0803988  accuracy=0.3942
Epoch=4   loss=2.0910833  accuracy=0.3979
Epoch=5   loss=2.0113904  accuracy=0.3992
Epoch=6   loss=2.1211505  accuracy=0.3898


KeyboardInterrupt: 

In [ ]:
'''
模型产生过拟合
'''